# Table tennis simulation

This example shows the usage of `Sim.jl` with event driven state machines.

In [1]:
using Sim, Printf
import Sim.init!

Here we implement players as nondeterministic finite automata (NFA's) and thus need some data describing the players and definitions of states and events:

In [2]:
abstract type PState end
struct Idle <: PState end
struct Wait <: PState end
struct Unalert <: PState end

abstract type PEvent end
struct Start <: PEvent end
struct Serve <: PEvent end
struct Return <: PEvent end
struct Miss <: PEvent end

mutable struct Player
    name::AbstractString
    sim::Union{Number, Clock}
    opp::Union{Number,Player}  # opponent
    state::PState              # state
    accuracy::Float64          # rate of accuracy for return
    attentiveness::Float64     # rate of attentiveness when hitting a ball
    score::Int64

    Player(name, acc, att) = new(name, 0, 0, Idle(), acc, att, 0)
end

Then we have some physical facts to define and a function to randomize them:

In [3]:
const dist = 3 # distance for ball to fly [m]
const vs   = 10 # serve velocity [m/s]
const vr   = 20 # return velocity [m/s]

rd(s::Float64) = randn()*s + 1

rd (generic function with 1 method)

Next we must describe the behaviour of our players. They are modeled as finite state machines, which have known states and react to known events. This is done with the `step!` function. Julia's multiple dispatch allows to give multiple definitions of `step!` for different combinations of states and events.

The `serve` and `ret`-functions, used for describing serving and return of players are used to randomize the time and the behaviour of players. The players thus act probabilistically as Markov automata.

In [4]:
"initialize a player"
function init!(p::Player, s::Clock, opp::Player)
    p.sim = s
    p.opp = opp
    p.state = rand() ≤ p.attentiveness ? Wait() : Unalert()
end

"a player serves a ball"
function serve(p::Player)
    ts = 3 + dist*rd(0.15)/(vs*rd(0.25))
    if rand() ≤ p.accuracy
        event!(p.sim, :(step!($(p.opp), Serve())), after, ts)
        @printf("%.2f: %s serves %s\n", p.sim.time+ts, p.name, p.opp.name)
    else
        event!(p.sim, :(step!($(p.opp), Miss())), after, ts)
        @printf("%.2f: %s serves and misses %s\n", p.sim.time+ts, p.name, p.opp.name)
    end
    if rand() ≥ p.attentiveness
        p.state = Unalert()
    end
end

"a player returns a ball"
function ret(p::Player)
    tr = dist*rd(0.15)/(vr*rd(0.25))
    if rand() ≤ p.accuracy
        event!(p.sim, :(step!($(p.opp), Return())), after, tr)
        @printf("%.2f: %s returns %s\n", p.sim.time+tr, p.name, p.opp.name)
    else
        event!(p.sim, :(step!($(p.opp), Miss())), after, tr)
        @printf("%.2f: %s returns and misses %s\n", p.sim.time+tr, p.name, p.opp.name)
    end
    if rand() ≥ p.attentiveness
        p.state = Unalert()
    end
end

"default transition for players"
step!(p::Player, q::PState, σ::PEvent) =
        println("undefined transition for $(p.name), $q, $σ")

"player p gets a start command"
step!(p::Player, ::Wait, ::Start) = serve(p)

"player p is waiting and gets served or returned"
step!(p::Player, ::Wait, ::Union{Serve, Return}) = ret(p)

"player p is unalert and gets served or returned"
function step!(p::Player, ::Unalert, ::Union{Serve, Return})
    @printf("%.2f: %s looses ball\n", p.sim.time, p.name)
    p.opp.score += 1
    p.state = Wait()
    serve(p)
end

"player p is waiting or unalert and gets missed"
function step!(p::Player, ::Union{Wait, Unalert}, ::Miss)
    p.score += 1
    p.state = Wait()
    serve(p)
end

"simplified `step!` call"
step!(p::Player, σ::PEvent) = step!(p, p.state, σ)

step!

In order to setup a simulation, we have to introduce a `Clock`, create and initialize the players, to start and run the game:

In [5]:
sim = Clock()
ping = Player("Ping", 0.90, 0.90)
pong = Player("Pong", 0.90, 0.90)
init!(ping, sim, pong)
init!(pong, sim, ping)
step!(ping, Start())

run!(sim, 30)
println("Ping scored $(ping.score)")
println("Pong scored $(pong.score)")

3.29: Ping serves Pong
3.46: Pong returns Ping
3.61: Ping returns and misses Pong
6.79: Pong serves Ping
6.96: Ping returns Pong
6.96: Pong looses ball
10.16: Pong serves Ping
10.38: Ping returns Pong
10.50: Pong returns Ping
10.66: Ping returns and misses Pong
14.13: Pong serves Ping
14.24: Ping returns Pong
14.38: Pong returns Ping
14.57: Ping returns Pong
14.71: Pong returns Ping
14.89: Ping returns Pong
15.02: Pong returns Ping
15.02: Ping looses ball
18.34: Ping serves and misses Pong
21.84: Pong serves Ping
22.11: Ping returns Pong
22.26: Pong returns Ping
22.38: Ping returns Pong
22.62: Pong returns Ping
22.75: Ping returns Pong
22.87: Pong returns Ping
22.98: Ping returns and misses Pong
26.37: Pong serves Ping
26.58: Ping returns and misses Pong
29.97: Pong serves Ping
30.09: Ping returns Pong
Finished: 28 events, simulation time: 30.0
Ping scored 1
Pong scored 6
